### 1. Create a s3 bucket

Using boto 3 -> its a AWS SDK for python which helps to crate,update and delete AWS resources form our python scripts

Boto3 makes it easy to integrate your python application,library or script with aws services including aws s3,EC2,dynamodb..

In [6]:
import boto3
s3 = boto3.resource('s3')

In [7]:
#creating a bucket
bucket_name = "yahoofinancestpricesagemaker"
try:
    s3.create_bucket(Bucket=bucket_name)
    print("s3 bucket has been created succesfully")
except Exception as e:
    print('S3 error:', e)

s3 bucket has been created succesfully


### 2. Create train and validation csv

In [18]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 2.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 51.7 MB/s eta 0:00:0000:0100:01
  Created wheel for peewee: filename=peewee-3.17.5-cp310-cp310-linux_x86_64.whl size=278850 sha256=7c839d331b411059a69c286b3790e2ff718d27b9fca0686ce2bef6c4ea7e8c52
  Stored in directory: /home/ec2-user/.cache/pip/wheels/06/80/9b/98db0d58349a2f5c09f8406789ade4270762f97b7d26f2fa22
Successfully built peewee


In [8]:
#extracting data about apple stock from 2023 to 2024
import pandas as pd
from datetime import datetime
import yfinance as yf

#initialize parameters
start_date = datetime(2019,1,1)
end_date = datetime(2021,1,1)

df_data = yf.download('AAPL',start=start_date,end=end_date)

df_data.reset_index(inplace=True)

#creating a dataframe
df_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,38.722500,39.712502,38.557499,39.480000,37.793781,148158800
1,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.029243,365248800
2,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.481926,234428400
3,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.402939,219111200
4,2019-01-08,37.389999,37.955002,37.130001,37.687500,36.077839,164101200
...,...,...,...,...,...,...,...
500,2020-12-24,131.320007,133.460007,131.100006,131.970001,129.339050,54930100
501,2020-12-28,133.990005,137.339996,133.509995,136.690002,133.964951,124486200
502,2020-12-29,138.050003,138.789993,134.339996,134.869995,132.181229,121047300
503,2020-12-30,135.580002,135.990005,133.399994,133.720001,131.054123,96452100


### 3. Extract load and transform the data

In [9]:
#dropping  Adj CLose and Date columns

df_data.drop(axis=1, columns=['Adj Close'],inplace=True)
df_data.drop(axis=1, columns=['Date'],inplace=True)

In [10]:
#Extracting features

#getiing all the features
df_data_features = df_data.iloc[:-1 , :]
df_data_features

,Open,High,Low,Close,Volume
0,38.722500,39.712502,38.557499,39.480000,148158800
1,35.994999,36.430000,35.500000,35.547501,365248800
2,36.132500,37.137501,35.950001,37.064999,234428400
3,37.174999,37.207500,36.474998,36.982498,219111200
4,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...
499,132.160004,132.429993,130.779999,130.960007,88223700
500,131.320007,133.460007,131.100006,131.970001,54930100
501,133.990005,137.339996,133.509995,136.690002,124486200
502,138.050003,138.789993,134.339996,134.869995,121047300


In [11]:
#now we will rename teh open price as targets
#here we are taking from the 2nd row
df_data_targets = df_data.iloc[1:, 0].rename("Targets")
df_data_targets

1       35.994999
2       36.132500
3       37.174999
4       37.389999
5       37.822498
          ...    
500    131.320007
501    133.990005
502    138.050003
503    135.580002
504    134.080002
Name: Targets, Length: 504, dtype: float64

In [12]:
#now combinng the Target adn the dataframe

df_data_features['Target'] = list(df_data_targets)

first_column = df_data_features.pop('Target')
df_data_features.insert(0,'Target',first_column)

df_data_final = df_data_features
df_data_final

/tmp/ipykernel_16641/2705682875.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_features['Target'] = list(df_data_targets)


,Target,Open,High,Low,Close,Volume
0,35.994999,38.722500,39.712502,38.557499,39.480000,148158800
1,36.132500,35.994999,36.430000,35.500000,35.547501,365248800
2,37.174999,36.132500,37.137501,35.950001,37.064999,234428400
3,37.389999,37.174999,37.207500,36.474998,36.982498,219111200
4,37.822498,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...,...
499,131.320007,132.160004,132.429993,130.779999,130.960007,88223700
500,133.990005,131.320007,133.460007,131.100006,131.970001,54930100
501,138.050003,133.990005,137.339996,133.509995,136.690002,124486200
502,135.580002,138.050003,138.789993,134.339996,134.869995,121047300


In [13]:
#note for xgboost to work we need to assign target as the first columns

### Train test split

In [14]:
import numpy as np
df_randomized = df_data_final.sample(frac=1, random_state=123)
df_randomized

#makinf the data ina random order for random selection

,Target,Open,High,Low,Close,Volume
429,115.230003,118.330002,118.830002,113.610001,115.540001,184642000
282,78.839996,81.184998,81.495003,80.712502,81.237503,80113600
383,97.264999,95.334999,95.980003,94.705002,95.919998,90257200
477,117.180000,118.639999,118.769997,117.290001,117.339996,73604300
428,118.330002,114.720001,115.930000,112.800003,115.360001,140150100
...,...,...,...,...,...,...
98,45.049999,44.950001,45.134998,44.452499,44.915001,146118800
476,118.639999,117.589996,119.059998,116.809998,118.639999,74113000
322,70.599998,70.000000,72.062500,69.512497,71.762497,194994800
382,95.334999,96.262497,96.317497,94.672501,95.752502,125642800


In [15]:
#splitting data basedon randomised data

train_data, test_data = np.split(df_randomized,[int(0.8*len(df_randomized))])
print(train_data.shape, test_data.shape)

(403, 6) (101, 6)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
### set path & upload dataset to s3

import os
prefix= 'xgboost-as-a-built-in-algo'

train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name,prefix,'train','train.csv')
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name,prefix,'test','test.csv')
print(train_csv_path)
print(test_csv_path)

s3://yahoofinancestpricesagemaker/xgboost-as-a-built-in-algo/train/train.csv
s3://yahoofinancestpricesagemaker/xgboost-as-a-built-in-algo/test/test.csv


In [18]:
train_data.to_csv(train_csv_path, index=False,header=False)
test_data.to_csv(test_csv_path, index=False,header=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


### 4. Build XGBoost Model

### How to use SageMaker XGBoost

we can use it as a framework and a built in algorithm
https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

we'll be using the XGboost as a built inalgorithm

In [2]:
import sagemaker
import boto3
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

### Find a xgboost image url and build a xgboost container

In [3]:
xgboost_container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.7-1")
display(xgboost_container)

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1'

## Initialize hyperparameters
### Booster parameters


        - max_depth: This parameter controls the maximum depth of a tree. Increasing this value makes 
                     the model more complex and  more likely to overfit. It's crucial for controlling 
                     the complexity of the model.

        - eta (learning rate): It controls the step size shrinkage used to prevent overfitting. After each 
                               boosting step, the eta parameter shrinks the feature weights to make the boosting
                               process more conservative. Lower values make the model more robust by shrinking 
                               the contribution of each tree.

        - gamma: This parameter specifies the minimum loss reduction required to make a further partition on 
                 a leaf node of the tree. It acts as a regularization parameter, controlling the complexity 
                 of the tree model.

        - min_child_weight: It defines the minimum sum of instance weight (hessian) needed in a child. 
                            In other words, it helps control over-fitting. Higher values prevent a model 
                            from learning relations which might be highly specific to the particular 
                            sample selected for a tree.

        - subsample: Denotes the fraction of observations to be randomly sampled for each tree.
                     Lower values make the algorithm more conservative and prevents overfitting
                     but setting it too low might lead to underfitting.

These parameters collectively help in fine-tuning the performance and generalization capability of the XGBoost model by controlling its complexity, learning aggressiveness, and robustness against noise in the training data. Adjusting these parameters requires balancing between model performance and overfitting, often through cross-validation or other validation techniques.

In [4]:
#settingthe hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:squarederror",
        "early_stopping_rounds":10,
        "num_round":1000}


### setting an output path where the trained model will be saved

In [10]:
output_path = 's3://{}/{}/{}/'.format(bucket_name,prefix,'output')
print(output_path)

s3://yahoofinancestpricesagemaker/xgboost-as-a-built-in-algo/output/


## construct a sagemaker estimator that calles the xgboost-container

    Enable the train_use_spot_instances constructor arg - a simple self-explanatory boolean
    
    set the train_max_wait constructor arg -this is an int arg representing the amount of time you are willing to wait for         the spot infrastructure to become available.Some instances types are harder to get at spot prices and you may have to wait 
    longer.you are not charged for the time spent once the spot instance have been successfully procured
    
    train_max_run - The timeout in seconds for training

In [11]:
#setting estimator with hyperparameters
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container,
                                          hyperparameters = hyperparameters,
                                          role = sagemaker.get_execution_role(),
                                          instance_count=1,
                                          instance_type='ml.m4.xlarge',
                                          volume_size =5, #5 GB,
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run = 300,
                                          max_wait = 600)

## Define the data type and paths to the training and validation datasets

In [12]:
content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name,prefix,'train'), content_type=content_type)
test_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name,prefix,'test'), content_type=content_type)

### Execute the XGBoost training job

In [13]:
estimator.fit({'train':train_input,'validation':test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-07-02-09-32-41-046


2024-07-02 09:32:41 Starting - Starting the training job...
2024-07-02 09:33:06 Starting - Preparing the instances for training......
2024-07-02 09:33:52 Downloading - Downloading input data...
2024-07-02 09:34:22 Downloading - Downloading the training image.........
2024-07-02 09:36:08 Training - Training image download completed. Training in progress.
2024-07-02 09:36:08 Uploading - Uploading generated training model[2024-07-02 09:35:59.311 ip-10-0-236-8.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-07-02 09:35:59.340 ip-10-0-236-8.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-07-02:09:35:59:INFO] Imported framework sagemaker_xgboost_container.training
[2024-07-02:09:35:59:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-07-02:09:35:59:INFO] No GPUs detected (normal if no gpus installed)
[2024-07-02:09:35:59:INFO] Running XGBoost Sagemaker in algorithm mo

In [14]:
#using spot intance helps to save cost

# 5. Deploy trained xgb model as endpoint

    1. Environment
       within sagemaker -serilization by User
       ##outside sagemaker  - serialization by Endpoint ***
   
    2. Method to invoke the endpoint
       ## API-single prediction
       s3 bucket -batch prediction
    
    3.Datatype based on method
       ## API - JSON
       s3 BUcket -csv

Using the json msg we'll trigger the json msg send to api gateway which 
via a lambda function will trigger the api endpoint

now that here the absed on the type of application it can be within or outside the sagemaker application

methods to invoke endpoints include single and batch prediction here we are using single prediction thus we make use of api



 NOw in order to host a model through AWS EC2 using amazon sagemaker,deploy the model that you trained and run a training job ny calling the #"deploy method" of the xgb_model_estimator

when you call the deploy method few key things that you need to specify

    Initial_instance_count(int)-- The number of instances to deploy the model

    instance_typr(str) - The type of instances that you want to operate your deployed model

    serializer(int) -Serialize the input data of various formats( a numpy array,list,file or buffer) to a csv formatted  string.we use this because the xgboost algorithm accepts the input files in csv format

In [15]:
from sagemaker.serializers import CSVSerializer
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge',serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-07-02-09-36-53-593
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-07-02-09-36-53-593
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-07-02-09-36-53-593


-------!

In [16]:
xgb_predictor.endpoint_name

'sagemaker-xgboost-2024-07-02-09-36-53-593'

### Make predictions with the use of Endpoints

In [19]:
import pandas as pd
from datetime import datetime
import yfinance as yf

#initialize parameters
start_date = datetime(2021,1,4)
end_date = datetime(2021,1,5)

df_data = yf.download('AAPL',start=start_date,end=end_date)

df_data.reset_index(inplace=True)

#creating a dataframe
df_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,133.520004,133.610001,126.760002,129.410004,126.83007,143301900


In [20]:
df_data.drop(axis=1, columns=['Adj Close'],inplace=True)
df_data.drop(axis=1, columns=['Date'],inplace=True)

data_features_array = df_data.values
data_features_array

array([[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,
        1.43301900e+08]])

# Serialize data

### Inference -Serialzed Input by Sagemaker Function

In [22]:
#to get the results
#if we dont use utf-8 then the output will be of bytes format
Y_pred_Fcn = xgb_predictor.predict(data_features_array).decode('utf-8')
Y_pred_Fcn

'132.52374267578125\n'

### Inference - Serialized Input by buily-in function (Lambda fucniton friendly) 

In [28]:
# input is a list of list to allow multiple row of features
Input = [[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,1.43301900e+08],
         [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,1.43301900e+08],
         [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,1.43301900e+08]]


#in lambda funciton we cannot use input serializer so

Serialized_Input = ','.join(map(str,Input[0]))

print(Serialized_Input, type(Serialized_Input))

133.520004,133.610001,126.760002,129.410004,143301900.0 <class 'str'>


In [29]:
Y_pred = xgb_predictor.predict(Serialized_Input).decode('utf-8')
Y_pred

'132.52374267578125\n'

In [26]:
# so in this session we learned how to create a endpoint ,serilize it and make it lambda friendly

## 6.0 Lambda funtion handler
   1)Trigger Endpoint 
   
   2)Trigger SNS

In [35]:
import boto3

ENDPOINT_NAME = 'sagemaker-xgboost-2024-07-02-09-36-53-593'
runtime = boto3.client('runtime.sagemaker')

def lambda_handler(event,context):
    inputs = event['data']  
    result=[]
    for inp in inputs:
        #serializing inputs
        serialized_input = ','.join(map(str,inp))
        response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                          ContentType='text/csv',
                                          Body=serialized_input)

    #we only need to get the body
        result.append(response['Body'].read().decode())
    return result



Input_json ={'data':
         [[1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,1.43301900e+08],
         [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,1.43301900e+08],
         [1.33520004e+02, 1.33610001e+02, 1.26760002e+02, 1.29410004e+02,1.43301900e+08]]
        }

lambda_handler(Input_json, __)

['132.52374267578125\n', '132.52374267578125\n', '132.52374267578125\n']